In [1]:
import carla
import math
import random
import time
import numpy as np
import cv2

client = carla.Client('127.0.0.1', 2003)
world = client.get_world()

bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [2]:
vehicle_bp = random.choice(bp_lib.filter('vehicle'))
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(
    vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)),
    vehicle.get_transform().rotation
)
spectator.set_transform(transform)


In [3]:
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(z=2.5, x=-4))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

scalar = 10
disp_size = [108, 108]*scalar

camera_bp.set_attribute('image_size_x', str(disp_size[0]))
camera_bp.set_attribute('image_size_y', str(disp_size[1]))
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))

In [4]:
camera.stop()

In [5]:
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    

In [6]:
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

camera_data ={'image': np.zeros((image_h, image_w, 4))}

camera.listen(lambda image: camera_callback(image, camera_data))

In [7]:
vehicle.set_autopilot(True)

In [9]:
cv2.namedWindow('RGB Camera', cv2.WINDOW_NORMAL)
cv2.resizeWindow('RGB Camera', 1280, 720)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)

while True:
    cv2.imshow('RGB Camera', camera_data['image'])
    
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()
    


: 